[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1buOH3HAiwWtofNl0P9lk-cSw-06nBpsC#scrollTo=v48AGvIfSW16)

### Step 1: Install Dependencies

This cell uses the `%pip` magic command to install the `flotorch` library. The `[adk]` extra specifies that all necessary dependencies for the Agent Development Kit (ADK), such as `google-adk`, should also be installed. This package provides the core tools for building and running your agent.

In [ ]:
%pip install flotorch[adk]

### Step 2: Configuration Variables

This cell defines the constant variables required to configure and authenticate the Flotorch agent.

* `FLOTORCH_API_KEY`: Replace the API key with your actual FloTorch API Key (Check [Prerequisites.ipynb](https://github.com/FloTorch/Resources/blob/main/examples/flotorch-rag-notebooks/faiss-example/01_Prerequisites.ipynb) on how to get the key)
                       
                       Using FloTorch helps you monitor the agentic flow through the FloTorch gateway.
* `FLOTORCH_BASE_URL`: The API gateway endpoint.
* `AGENT_NAME`: A unique name for this specific agent instance, indicating it uses MCP (Multi-turn Conversation Protocol) tools.
* `APP_NAME`: A name for your application, used to group sessions.
* `USER_ID`: A unique identifier for the end-user interacting with the agent.

In [ ]:
from getpass import getpass

FLOTORCH_API_KEY = getpass("FloTorch API Key") # SET YOUR FLOTORCH API KEY HERE
FLOTORCH_BASE_URL = "https://gateway.flotorch.cloud" 
AGENT_NAME = "single-agent-with-local-tool"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

### Step 3: Import Libraries

This cell imports all the necessary classes and modules from the Flotorch and Google ADK libraries. These components are the building blocks for creating the agent (`FlotorchADKAgent`), managing conversation history (`FlotorchADKSession`), defining tools (`FunctionTool`), and running the agent (`Runner`).

In [ ]:
# Import necessary libraries
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types
from google.adk.tools import FunctionTool

print("Imported necessary libraries successfully")

### Step 4: Define a Custom Tool (Google Search)

This cell defines a custom tool that the agent can use to answer questions.

1.  A Python function `Google Search` is created. It takes a `query` string and uses the `requests` library to call the Google Custom Search API.
2.  It parses the JSON response and formats the top 5 results into a single, easy-to-read string.
3.  The function is then wrapped in the ADK's `FunctionTool` class, which allows the agent's underlying language model to understand what the tool does and how to call it.
4.  This `FunctionTool` object is stored in a list called `custom_tools`, which will be passed to the agent during initialization.

In [ ]:
import requests
from google.adk.tools import FunctionTool

# --- Google Search Tool ---
def google_search(query: str) -> str:
    """Perform a Google search and return top results."""
    api_key = "" # get your google api key
    cse_id = ""  # get your google cse_id key

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": api_key,
        "cx": cse_id,
        "q": query,
        "num": 5  # top 5 results
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "items" not in data:
        return "No results found."

    results = []
    for item in data["items"]:
        title = item.get("title", "")
        snippet = item.get("snippet", "")
        link = item.get("link", "")
        results.append(f"🔹 {title}\n{snippet}\n{link}")

    # Combine results into a single text output
    return "\n\n".join(results)

# --- Wrap as ADK Tool ---
custom_tools = [FunctionTool(func=google_search)]

print("✅ Google Search tool created successfully.")


### Step 5: Initialize Session Service

This cell initializes the `FlotorchADKSession` service. This service is responsible for creating, retrieving, and managing conversation sessions (i.e., keeping track of the chat history). It's configured using the API key and base URL defined in step 2. Note that the `FlotorchMemoryService` initialization is commented out in this example.

In [ ]:
# Initialize Session
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY, base_url=FLOTORCH_BASE_URL
)

print("Initialized Session")

### Step 6: Initialize the Flotorch Agent

Here, the `FlotorchADKAgent` is initialized. This is the core of the application. It is configured with its name, API credentials, and (most importantly) the `custom_tools` list from step 4. This step makes the agent "aware" of the `Google Search` function and gives it the ability to decide when to use it. The `.get_agent()` method retrieves the final, configured agent object.

In [ ]:
flotorch_client = FlotorchADKAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools=custom_tools
)

agent = flotorch_client.get_agent()
print(f"Advanced FlotorchADKAgent '{agent.name}' created.")

### Step 7: Configure the Runner and Chat Function

This cell does two key things:

1.  **Initializes the `Runner`**: The `Runner` from the `google.adk` library is the component that orchestrates the entire conversation flow. It connects the `agent` (step 6) with the `session_service` (step 5) to process messages.
2.  **Defines `chat_with_agent`**: This is an asynchronous helper function that simplifies the process of interacting with the agent. It takes a `query` and `session_id`, sends the message to the `runner`, processes all the turn-based events, and returns only the agent's final text response.

In [ ]:
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

async def chat_with_agent(query,session_id):
    """
    Chat function that returns the agent's response and saves the interaction to memory.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                # Save the session to memory after getting a response
                completed_session = await runner.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=session_id)
                # await memory_service.add_session_to_memory(completed_session)
                return event.content.parts[0].text
    return "Sorry, I couldn't process that request."

print("Comprehensive runner configured successfully.")

### Step 8: Run the Conversation

This is the main execution cell where the agent is tested.

1.  It first creates a new session (`session1`) using the `session_service`.
2.  It then calls `chat_with_agent` with the query "what is stock price of tata motors?". The agent will see this query, decide it needs external information, and call its `Google Search` tool.
3.  The agent's final response (based on the search results) is printed.
4.  It calls `chat_with_agent` again *in the same session* with a new query: "what is current temprature in hyderabad?".
5.  The agent repeats the process, calling the `Google Search` tool again to find the weather. The final response is printed.

In [ ]:
# === Session 1: Teach the agent a fact and use a tool ===
session1 = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
print(f"Started Session 1 with ID: {session1.id}\n")
response1 = await chat_with_agent("what is stock price of tata motors?", session1.id)
print("user: what is stock price of tata motors")
print(f"Response: {response1}")

response2 = await chat_with_agent(" what is current temperature in hyderabad?", session1.id)
print(f"\nQuery: what is current temperature in hyderabad? ")
print(f"Response: {response2}")

print("--- Session 1 ended and memory saved ---")

